In [ ]:
from itertools import chain
from types import MappingProxyType
import os

# chances are you're running this from a directory above /src
os.chdir('..')

from dateutil.parser import parse

from aws_utils.ssh import *
from bulk_conversion_notebooks.m3_bulk import basenamer, make_m3_triplet, \
    m3_triplet_bundle_paths, crude_time_log, fix_end_object_tags
from m3_conversion import *
from mtools.mweb import filename_from_path
from pvl.decoder import ParseError

In [ ]:
# where are our buckets mounted?
input_directory = './remote/m3_input/'
output_directory = './remote/m3_output/'

In [ ]:
browse_files = []
for av in range(3):
    directory = input_directory + 'CH1M3_000' + str(av+1) + '/EXTRAS/BROWSE/'
    dir_files = os.listdir(directory)
    browse_files += [directory + file for file in dir_files]
browse_files = pd.DataFrame([file for file in browse_files if '.JPG' in file], columns=['path'])
browse_files['fn'] = browse_files['path'].apply(filename_from_path)
browse_files['basename'] = browse_files["fn"].apply(basenamer)
unique_browse = browse_files.drop(
    browse_files.loc[browse_files.duplicated(subset="fn")].index
)

In [ ]:
browse_groups = list(unique_browse.groupby("basename"))
global_groups = [
    group[1] for group in browse_groups
    if 'M3G' in group[0] 
]
target_groups = [
    group[1] for group in browse_groups
    if 'M3T' in group[0] 
]

In [ ]:
assert all([len(group)==2 for group in global_groups])
assert all([len(group)==4 for group in target_groups])

In [ ]:
for group in global_groups:
    albedo_images = reloc(group,'fn','B046')
    thermal_images = reloc(group, 'fn', 'B084')
    assert len(albedo_images) == 1
    assert len(thermal_images) == 1
    time = parse(interpret_m3_basename(group['fn'].iloc[0])['time'])
    year = str(time.year)
    month = str(time.month)
    day = str(time.day)
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    image_output_directory = output_directory + 'browse/'+year+month+'/'+year+month+day+'/' 
    sh.mkdir("-p", image_output_directory)
    albedo_writer = M3BrowseWriter(
        albedo_images['path'].iloc[0]
    )
    thermal_writer = M3BrowseWriter(
        thermal_images['path'].iloc[0]
    )
    albedo_writer.write_pds4(image_output_directory)
    thermal_writer.write_pds4(image_output_directory)

In [ ]:
for group in target_groups:
    albedo_images = reloc(group,'fn','(B046|B084|B105)')
    thermal_images = reloc(group, 'fn', '(B250)')
    assert len(albedo_images) == 3
    assert len(thermal_images) == 1
    time = parse(interpret_m3_basename(group['fn'].iloc[0])['time'])
    year = str(time.year)
    month = str(time.month)
    day = str(time.day)
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    image_output_directory = output_directory + 'browse/'+year+month+'/'+year+month+day+'/' 
    sh.mkdir("-p", image_output_directory)
    albedo_writer = M3BrowseWriter(
        albedo_images['path'].iloc[0],
        second_image_path = albedo_images['path'].iloc[1],
        third_image_path = albedo_images['path'].iloc[2]
    )
    thermal_writer = M3BrowseWriter(
        thermal_images['path'].iloc[0],
    )
    albedo_writer.write_pds4(image_output_directory)
    thermal_writer.write_pds4(image_output_directory)

In [ ]:
validate_results = sh.validate("-t", './remote/m3_output/browse/')

In [ ]:
validate_results